In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import utilities as ut



In [2]:
# DEBEMOS RECIBIR EL ID DE USUARIO COMO PARAMETRO DE ENTRADA Y EL NUMERO DE COINCIDENCIAS BUSCADAS
ID_usuario = 1
Top = 10

In [3]:
database=pd.read_csv("database_corregido.csv")

In [4]:
# Ejemplo de recomendación de usuarios

def RecomendacionUsuario(Top, ID_usuario):
    ID_usuario = ID_usuario - 1
    df = database.copy()
    RecomendacionUsuario = ut.carga_datos(database).iloc[(-ut.carga_datos(database).iloc[:, ID_usuario]).argsort()[1:(Top+1)].values, ID_usuario]
    RecomendacionUsuario.index

    filtrado = pd.DataFrame()
    for id in RecomendacionUsuario.index:
        filtrado = filtrado.append(df[df["user_id"]== id])
    return filtrado



In [5]:
RecomendacionUsuario(10, ID_usuario)

,user_id,user_name,usersurname,email,password,year_birth,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
371,372,Mary,Ellen,ohegla@gmail.com,WnyLRGint,1989,Masculino,"Frances, Rumano",De 1 año a 2 años,Sin graduado escolar,No,Emocional,No,Valle del Jarama,"Naturaleza, Deportes, Juegos",01/08/2021,NaN,NaN
157,158,Louis,Archie,uacakzbf@gmail.com,POIVdWegI,1991,Femenino,Frances,Menos de 6 meses,Doctorado,Si,Emocional,No,Valle del Jarama,"Cocina, Juegos, Baile",08/02/2022,NaN,NaN
87,88,Shannon,Mary,wg@yahoo.com,zMWxmVJox,1982,No especificar,"Chino, Español",Más de 2 años,Sin graduado escolar,No,Orientacion sobre tramites,Si,Valle del Jarama,"Juegos, Musica, Viajar",15/12/2021,NaN,NaN
321,322,Monica,Thelma,q@gmail.com,LPvJgJOTc,1988,Masculino,Chino,Más de 2 años,Sin graduado escolar,No,Orientacion laboral,No,Valle del Jarama,"Deportes, Viajar, Cocina",13/08/2022,NaN,NaN
6,7,Kenneth,Jenny,ybvgmhuyz@aol.com,oiFAMfhta,1997,Femenino,"Italiano, Chino",De 6 meses a 1 año,Sin graduado escolar,No,Orientacion laboral,No,Valle Medio del Lozoya,"Musica, Baile, Naturaleza",13/04/2022,NaN,NaN
685,686,Larry,Beatrice,jvcsgtav@gmail.com,WSdnZvvkR,1988,Masculino,"Rumano, Frances",Menos de 6 meses,Sin graduado escolar,No especificar,Orientacion sobre tramites,No,Valle Bajo del Lozoya,"Juegos, Baile, Deportes",01/04/2021,NaN,NaN
114,115,Oscar,Joseph,h@outlook.com,rIrjSeWYI,1988,No especificar,"Portugues, Rumano",Menos de 6 meses,Sin graduado escolar,No,Orientacion sobre tramites,Si,Valle del Jarama,"Juegos, Viajar, Baile",30/12/2021,NaN,NaN
741,742,Consuelo,Dawn,l@yahoo.com,yrRXEbhlo,2001,No especificar,"Frances, Italiano",De 6 meses a 1 año,Sin graduado escolar,Si,Orientacion laboral,Si,Sierra del Rincon,"Baile, Musica, Juegos",02/03/2022,NaN,NaN
60,61,John,Anthony,istiaxp@gmail.com,TqtWiYWUg,1999,Masculino,Chino,Más de 2 años,Sin graduado escolar,Si,Orientacion sobre tramites,Si,Valle del Jarama,"Baile, Fotografia, Viajar",13/05/2022,NaN,NaN
586,587,Jody,Lonnie,enwaqpmna@aol.com,bnyRCDJQC,2001,No especificar,Chino,De 1 año a 2 años,Sin graduado escolar,Si,Orientacion laboral,Si,Valle del Jarama,"Baile, Cine, Fotografia",02/06/2021,NaN,NaN


In [8]:
def RecomendacionExperto(Top, ID_usuario):
    df = ut.automatizacion(ID_usuario, database).copy()
    
    RecomendacionUsuario = ut.carga_datos_expertos(df)[ID_usuario].sort_values(ascending=False)[1:Top]
    RecomendacionUsuario.index

    filtrado_expertos = pd.DataFrame()
    for id in RecomendacionUsuario.index:
        filtrado_expertos = filtrado_expertos.append(df[df["user_id"]== id])
    return filtrado_expertos

In [9]:
RecomendacionExperto(Top, ID_usuario)

,user_id,user_name,usersurname,email,password,year_birth,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
839,840,Danielle,Amanda,alxnsluq@mail.com,FMdnfQCRm,1999,No especificar,Frances,Menos de 6 meses,Master,Si,Emocional,Si,Valle Medio del Lozoya,"Pintura, Baile, Cine",23/12/2022,NaN,NaN
282,283,Lionel,Daniel,owjrnhu@mail.kz,JERpUzIph,1966,No especificar,"Frances, Portugues",De 6 meses a 1 año,FP (Formacion Profesional),No,Emocional,Si,Sierra del Rincon,"Lectura, Musica, Cocina",02/03/2021,NaN,NaN
373,374,Valerie,Betty,f@yahoo.com,lIAQdIszK,1967,Masculino,"Aleman, Chino",Menos de 6 meses,Grado universitario,No,Emocional,Si,Valle Medio del Lozoya,"Deportes, Juegos, Cocina",28/02/2022,NaN,NaN
409,410,Christie,Dan,yskldjhfd@hotmail.com,LXduvXfpe,1964,Femenino,"Chino, Rumano",Menos de 6 meses,Doctorado,No,Emocional,Si,Valle Medio del Lozoya,"Deportes, Cocina, Pintura",02/12/2021,NaN,NaN
257,258,Bryan,Richard,wimaberql@mail.com,BsvlIdfaH,1980,Femenino,"Chino, Aleman",Menos de 6 meses,Master,No,Emocional,Si,Valle Alto del Lozoya,"Naturaleza, Juegos, Deportes",12/05/2022,NaN,NaN
762,763,Patricia,Vincent,bp@hotmail.com,CRpYxvIuI,1973,Masculino,Frances,Menos de 6 meses,Grado universitario,No especificar,Emocional,Si,Sierra del Rincon,"Juegos, Musica, Baile",04/01/2022,NaN,NaN
48,49,Catherine,Eileen,f@aol.com,HiuTFTNia,1977,Masculino,"Chino, Ingles",Menos de 6 meses,Con graduado escolar,No especificar,Emocional,Si,Valle Alto del Lozoya,"Pintura, Baile, Musica",28/02/2022,NaN,NaN
193,194,Clifford,Frances,v@aol.com,riEUIzjID,1999,Femenino,"Chino, Ucraniano",Menos de 6 meses,Con graduado escolar,No especificar,Emocional,Si,Sierra de la Cabrera,"Juegos, Deportes, Naturaleza",12/02/2021,NaN,NaN
513,514,Don,Heidi,lp@mail.com,FtxaADcuo,1970,No especificar,"Italiano, Chino",Menos de 6 meses,FP (Formacion Profesional),Si,Emocional,Si,Valle del Jarama,"Viajar, Cocina, Lectura",19/01/2022,NaN,NaN
625,626,Sylvia,Hui,owkdc@aol.com,hUQidmMgJ,1971,Femenino,"Italiano, Chino",De 1 año a 2 años,Master,No especificar,Emocional,Si,Valle del Jarama,"Pintura, Naturaleza, Cocina",25/12/2022,NaN,NaN
